In [1]:
# Install lap, required for SFSORT
!pip install lap

#Install ultralytics to access the object detector
#!pip install ultralytics

  Using cached lap-0.4.0.tar.gz (1.5 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for lap
Failed to build lap


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [28 lines of output]
      C:\Users\dnnxl\AppData\Local\Temp\pip-install-1kb73ypc\lap_5b7d075577954d648b6f3a6c1e325195\setup.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
        from pkg_resources import parse_version
      Partial import of lap during the build process.
      C:\Users\dnnxl\AppData\Local\Temp\pip-install-1kb73ypc\lap_5b7d075577954d648b6f3a6c1e325195\setup.py:223: DeprecationWarning:
      
        `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
        of the deprecation of `distutils` itself. It will be removed for
        Python >= 3.12. For older Python versions it will remain present.
        It is recommended to use `setuptools < 60.0` for those Python versions.
        For more details, see:
          https://numpy.org/devdocs/reference/distutils_s

In [1]:
# Import essential libraries
import numpy as np
import cv2

from ultralytics import YOLO
from ultralytics.utils.torch_utils import select_device
from random import randrange

from SFSORT import SFSORT

WARNING  Python>=3.10 is required, but Python==3.8.18 is currently installed 


In [2]:
# Instantiate an object detector
model = YOLO('C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/yolo11n_DJI_0008_V_and_0010_V_2@fine-tuning/best.pt', 'detect')

# Check for GPU availability
device = select_device('cpu')
# Devolve the processing to selected devices
model.to(device)

Ultralytics 8.3.11  Python-3.8.18 torch-2.1.0 CPU (12th Gen Intel Core(TM) i5-1235U)


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [3]:
import os

def list_files(directory):
    try:
        # List all files in the given directory
        files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
        return files
    except Exception as e:
        return str(e)  # Return the error message as a string

def list_directories(path='.'):
    return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

path_mot_dataset = 'C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308111117_0010_V_1/valid/images'
files_in_directory = [os.path.join(path_mot_dataset, file) for file in list_files(path_mot_dataset)]

In [4]:
import cv2
import re
from collections import defaultdict

import numpy as np

frame_rate = 25 
frame_width = 1920
frame_height = 1088
# Organize tracker arguments into standard format
tracker_arguments = {"dynamic_tuning": True, "cth": 0.7,
                      "high_th": 0.82, "high_th_m": 0.1,
                      "match_th_first": 0.5, "match_th_first_m": 0.05,
                      "match_th_second": 0.1, "low_th": 0.3,
                      "new_track_th": 0.7, "new_track_th_m": 0.1,
                      "marginal_timeout": (7 * frame_rate // 10),
                      "central_timeout": frame_rate,
                      "horizontal_margin": frame_width // 10,
                      "vertical_margin": frame_height // 10,
                      "frame_width": frame_width,
                      "frame_height": frame_height}

#track_history = defaultdict(lambda: [])
track_history = []

# Instantiate a tracker
tracker = SFSORT(tracker_arguments)

#fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#out = cv2.VideoWriter('output.mp4', fourcc, 25.0, (frame_width, frame_height))

# Define a color list for track visualization
colors = {}

for file in files_in_directory:
    match = re.search(r'frame_\d+\.png', file)
    if match:
        frame_file = match.group(0)
        frame_id = int(frame_file.split(".")[0].split("_")[1])
        
    frame = cv2.imread(file)
    #results = model.track(frame, persist=True, tracker="botsort.yaml")
    prediction = model.predict(frame)

    # Exclude additional information from the predictions
    prediction_results = prediction[0].boxes.cpu().numpy()

    # Update the tracker with the latest detections
    tracks = tracker.update(prediction_results.xyxy, prediction_results.conf)

    if len(tracks) == 0:
      continue

    # Extract tracking data from the tracker
    bbox_list = tracks[:, 0]
    track_id_list = tracks[:, 1]

    # Visualize tracks
    for idx, (track_id, bbox) in enumerate(zip(track_id_list, bbox_list)):
    
      if track_id not in colors:
        colors[track_id] = (randrange(255), randrange(255), randrange(255))

      # Extract the bounding box coordinates
      x0, y0, x1, y1 = map(int, bbox)

      track_history.append((frame_id, track_id, x0, y0, x1-x0, y1-y0, -1, -1,-1,-1))

      # Draw the bounding boxes on the frame
      color = colors[track_id]

      annotated_frame = cv2.rectangle(frame, (x0, y0), (x1, y1), color, 2)
      # Put the track label on the frame alongside the bounding box
      cv2.putText(annotated_frame, str(track_id), (x0, y0-5),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    # Save the annotated frame as an image
    cv2.imwrite(f'annotated_frames/frame_{frame_id}.jpg', annotated_frame)
    #out.write(annotated_frame)
#out.release()

c:\Users\dnnxl\anaconda3\envs\sam-pinas-dev\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0: 384x640 1 pineapple, 146.6ms
Speed: 4.3ms preprocess, 146.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 120.6ms
Speed: 6.8ms preprocess, 120.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 92.8ms
Speed: 1.0ms preprocess, 92.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pineapple, 91.7ms
Speed: 3.0ms preprocess, 91.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pineapple, 76.6ms
Speed: 2.0ms preprocess, 76.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 79.8ms
Speed: 0.0ms preprocess, 79.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 124.3ms
Speed: 2.0ms preprocess, 124.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 101.1ms
Speed: 0.0ms preprocess, 101.1ms infere

In [7]:
# Function to write MOT data to a text file
def write_mot_format(data, filename="sfsort_mot.txt"):
    with open(filename, "w") as f:
        for entry in data:
            # Formatting each entry as per MOT challenge format (one line per object)
            line = "{},{},{},{},{},{},{},{},{},{}\n".format(*entry)
            f.write(line)

# Call the function to write to file
write_mot_format(track_history)